<a href="https://colab.research.google.com/github/T-K-O-H/youtube_to_linkedin/blob/main/full_model_finetuning_wandb_cosine_amp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔧 Full Model Fine-Tuning with W&B, Cosine LR, and AMP
This notebook fine-tunes a Sentence Transformer model to improve query-to-transcript chunk retrieval using `MultipleNegativesRankingLoss`. Includes logging with Weights & Biases and uses mixed precision.

In [ ]:
# ✅ Install dependencies
!pip install sentence-transformers wandb huggingface-hub datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 📚 Imports
import json
import wandb
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader

In [ ]:
train_samples = []
with open("/content/data.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue  # skip blank lines
        try:
            item = json.loads(line)
            query = item["query"]
            chunk = item["chunk"]
            train_samples.append(InputExample(texts=[query, chunk]))
        except json.JSONDecodeError:
            print("Skipping invalid JSON line:", line)

In [ ]:
model_name = "sentence-transformers/all-mpnet-base-v2"
word_embedding_model = models.Transformer(model_name, max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
# 🔄 DataLoader and loss function
from sentence_transformers.losses import MultipleNegativesRankingLoss, MatryoshkaLoss

dataloader_1 = DataLoader(train_samples, batch_size=8, shuffle=True, drop_last=True)
dataloader_2 = DataLoader(train_samples, batch_size=8, shuffle=True, drop_last=True)


loss_1 = MultipleNegativesRankingLoss(model)

loss_2 = MatryoshkaLoss(
    model=model,
    loss=MultipleNegativesRankingLoss(model),  # nested loss
    matryoshka_dims=[768, 512, 256, 128]
)

In [ ]:
# 🧪 Initialize W&B for tracking
wandb.login()
wandb.init(project="transcript-retrieval", name="full-model-finetuned", mode="online")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
# --- Setup ---
import wandb
import json
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.losses import MultipleNegativesRankingLoss, MatryoshkaLoss
from torch.utils.data import DataLoader
from torch.optim import AdamW

# --- WandB Init ---
wandb.login(relogin=True)
wandb.init(
    project="mpnet-finetune",
    name="matryoshka-mnr-combined",
    config={
        "epochs": 5,
        "lr": 2e-5,
        "scheduler": "warmupcosine",
        "batch_size": 32,
        "loss": "Matryoshka + MultipleNegatives",
        "base_model": "all-mpnet-base-v2"
    }
)

# --- Simple WandB Logger ---
class WandbLogger:
    def __init__(self, optimizer=None, evaluator=None):
        self.optimizer = optimizer
        self.evaluator = evaluator

    def __call__(self, score, epoch, steps):
        log_data = {
            "epoch": epoch,
            "loss": score,
            "steps": steps
        }

        if self.optimizer:
            log_data["learning_rate"] = self.optimizer.param_groups[0]['lr']

        if self.evaluator:
            val_score = self.evaluator()
            log_data["val_score"] = val_score

        wandb.log(log_data)

# --- Load training data ---
with open("/content/data.jsonl", "r", encoding="utf-8") as f:
    train_samples = [json.loads(line) for line in f]

samples = [InputExample(texts=[s["query"], s["chunk"]]) for s in train_samples]
dataloader_1 = DataLoader(samples, shuffle=True, batch_size=32, drop_last=True)
dataloader_2 = DataLoader(samples, shuffle=True, batch_size=32, drop_last=True)

# --- Load model ---
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# --- Create optimizer ---
optimizer = AdamW(model.parameters(), lr=2e-5)

# --- Define losses ---
loss_1 = MultipleNegativesRankingLoss(model)
loss_2 = MatryoshkaLoss(
    model=model,
    loss=MultipleNegativesRankingLoss(model),
    matryoshka_dims=[768, 512, 256, 128]
)

# --- Train the model ---
model.fit(
    train_objectives=[(dataloader_1, loss_1), (dataloader_2, loss_2)],
    epochs=5,
    warmup_steps=100,
    optimizer_params={"lr": 2e-5},
    scheduler="warmupcosine",
    show_progress_bar=True,
    use_amp=True,
    output_path="finetuned_mpnet_matryoshka_mnr",
    callback=WandbLogger(optimizer=optimizer)
)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


train/epoch,▁█
train/global_step,▁█
train/grad_norm,█▁
train/learning_rate,█▁
train/loss,█▁
train/epoch,0.3858
train/global_step,1000
train/grad_norm,0.6156
train/learning_rate,2e-05
train/loss,0.0375


Computing widget examples:   0%|          | 0/2 [00:00<?, ?example/s]

Step,Training Loss
500,0.087800
1000,0.037500
1500,0.029000
2000,0.023800
2500,0.014300
3000,0.008600
3500,0.006300
4000,0.006800
4500,0.005000
5000,0.004900


In [ ]:
from getpass import getpass
from huggingface_hub import login

hf_token = getpass("Enter your Hugging Face token: ")
login(token=hf_token)

Enter your Hugging Face token: ··········


In [ ]:
from huggingface_hub import HfApi, HfFolder
from sentence_transformers import SentenceTransformer

# Load the fine-tuned model
model = SentenceTransformer("finetuned_mpnet_matryoshka_mnr")

# Push to Hugging Face (change user/model_id)
model.push_to_hub("Shipmaster1/finetuned_mpnet_matryoshka_mnr")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/Shipmaster1/finetuned_mpnet_matryoshka_mnr/commit/f50d67da79d8d90ba243442d0989144f79ea4408'